In [ ]:
import pickle
import numpy as np

participantCount = 32

rawData = [{}]*participantCount

for i in range(participantCount):
    rawData[i] = pickle.load(open(f'../data/s{i+1:02}.dat', 'rb'), encoding="latin1")

labels = np.array(list(map(lambda participant : participant['labels'], rawData)))
data = np.array(list(map(lambda participant : participant['data'], rawData)))

def get_y(emotion):
    return (labels.reshape(-1, 4)[:,emotion] >= 5).astype(int)

def get_peripheral_x():
    return data[:, :, 32:, :].reshape(-1, 8, 8064)

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.model_selection import KFold
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

Y = get_y(0)
X = np.moveaxis(get_peripheral_x(), 1, 2)

for i in range(4):
    print("EMOTION NUMBER", i)
    Y = get_y(i)
    kfold = KFold(n_splits=5, shuffle=False)
    cvscores = []
    for train, test in kfold.split(X, Y):
        model = Sequential()
        model.add(LSTM(100, input_dim=8))
        model.add(Dense(1, activation='sigmoid'))
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["acc", f1_m, precision_m, recall_m])

        model.fit(X[train], Y[train], validation_data=(X[test], Y[test]), epochs=20, batch_size=120, verbose=2, shuffle=False)

        scores = model.evaluate(X[test], Y[test], verbose=0)
        print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
        cvscores.append(scores[1] * 100)

    print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))    
    print("EMOTION", i)